[View in Colaboratory](https://colab.research.google.com/github/Gangsss/BOAZ_NLP_STUDY/blob/master/03_GloVe.ipynb)

# 3. GloVe: Global Vectors for Word Representation

I recommend you take a look at these material first.

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture3.pdf
* https://nlp.stanford.edu/pubs/glove.pdf

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [38]:
print(torch.__version__)
print(nltk.__version__)

0.3.0.post4
3.2.5


In [0]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
torch.cuda.set_device(gpus[0])

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [0]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [0]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

## Data load and Preprocessing 

In [42]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500]
# 대문자 -> 소문자 처리
corpus = [[word.lower() for word in sent] for sent in corpus]

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Build vocab

In [43]:
vocab = list(set(flatten(corpus)))
print(vocab[0:5])

['10', 'fates', '1690', 'boots', 'though']


In [0]:
word2index = {}
# word2index - word : index 형태
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
# index2word - index : word 형태로 변경
index2word={v:k for k, v in word2index.items()}

In [45]:
WINDOW_SIZE = 5
windows =  flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])
print(windows[0])

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville')


In [46]:
window_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': 
            continue
        window_data.append((window[WINDOW_SIZE], window[i]))

print(window_data[0:5])


[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('[', 'herman'), ('[', 'melville')]


### Weighting Function 

<img src="../images/03.glove-weighting-function.png">
<center>borrowed image from https://nlp.stanford.edu/pubs/glove.pdf</center>

In [0]:
def weighting(w_i, w_j):
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1
        
    x_max = 100 #100 # fixed in paper
    alpha = 0.75
    
    if x_ij < x_max:
        result = (x_ij/x_max)**alpha
    else:
        result = 1
    
    return result

### Build Co-occurence Matrix X

Because of model complexity, It is important to determine whether a tighter bound can be placed on the number of nonzero elements of X.

In [0]:
X_i = Counter(flatten(corpus)) # X_i
#각 단어의 count 수

In [52]:
X_ik_window_5 = Counter(window_data) # Co-occurece in window size 5
# 같이 등장한 쌍들의 갯수 
print(list(X_ik_window_5)[0:5])

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('[', 'herman'), ('[', 'melville')]


In [63]:
print(X_ik_window_5)

Counter({(',', ','): 342, (',', 'the'): 280, ('the', ','): 280, ('the', 'of'): 248, ('of', 'the'): 248, (',', 'and'): 229, ('and', ','): 229, ('the', 'the'): 220, (',', 'a'): 167, ('a', ','): 167, ('of', ','): 164, (',', 'of'): 164, ('in', ','): 125, (',', 'in'): 125, ('to', ','): 121, (',', 'to'): 121, ('the', 'and'): 121, ('and', 'the'): 121, ('the', 'in'): 113, ('in', 'the'): 113, ("'", 's'): 92, ('s', "'"): 92, ('of', 'a'): 80, ('a', 'of'): 80, ('the', '.'): 79, ('.', 'the'): 79, ('to', 'the'): 75, ('the', 'to'): 75, (',', 'i'): 74, ('i', ','): 74, ('the', 'whale'): 69, ('whale', 'the'): 69, ('the', 'a'): 68, ('a', 'the'): 68, ('-', ','): 67, (',', '-'): 67, ('and', 'a'): 65, ('a', 'and'): 65, (',', 'that'): 65, ('that', ','): 65, ('and', 'of'): 64, ('of', 'and'): 64, ('.', ','): 58, (',', '.'): 58, ('a', 'to'): 57, ('to', 'a'): 57, ('in', 'a'): 56, ('a', 'in'): 56, ('.', '.'): 54, ('the', '-'): 54, ('-', 'the'): 54, ('a', '.'): 53, ('.', 'a'): 53, ('"', 'the'): 53, ('the', '"'): 5

In [60]:
X_ik_window_5[('.', '.')]

54

In [0]:
X_ik = {}
weighting_dic = {}

In [0]:
from itertools import combinations_with_replacement
#combinations_with_replacement('ABCD', 2) ->	 	AA AB AC AD BB BC BD CC CD DD

In [55]:
vocab_c= list(combinations_with_replacement(vocab, 2))
print(vocab_c[0:10])

[('10', '10'), ('10', 'fates'), ('10', '1690'), ('10', 'boots'), ('10', 'though'), ('10', 'whatsoever'), ('10', 'cheever'), ('10', 'separate'), ('10', 'fireplaces'), ('10', 'objectionable')]


In [0]:
def weighting(w_i, w_j):
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1
        
    x_max = 100 #100 # fixed in paper
    alpha = 0.75
    
    if x_ij < x_max:
        result = (x_ij/x_max)**alpha
    else:
        result = 1
    
    return result

In [0]:
for bigram in combinations_with_replacement(vocab, 2): 
    if X_ik_window_5.get(bigram) is not None: # nonzero elements
        co_occer = X_ik_window_5[bigram] # 동시등장 횟수
        X_ik[bigram] = co_occer + 1 # log(Xik) -> log(Xik+1) to prevent divergence
        X_ik[(bigram[1],bigram[0])] = co_occer+1
    else:
        pass
        
    weighting_dic[bigram] = weighting(bigram[0], bigram[1])
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0])

In [62]:
print(X_ik)

{('10', 'or'): 2, ('or', '10'): 2, ('10', 'to'): 2, ('to', '10'): 2, ('10', 'altogether'): 2, ('altogether', '10'): 2, ('10', 'nearly'): 2, ('nearly', '10'): 2, ('10', 'yards'): 2, ('yards', '10'): 2, ('10', 'amounted'): 2, ('amounted', '10'): 2, ('10', '440'): 2, ('440', '10'): 2, ('10', ','): 3, (',', '10'): 3, ('10', 'whale'): 2, ('whale', '10'): 2, ('fates', 'officer'): 2, ('officer', 'fates'): 2, ('fates', 'the'): 4, ('the', 'fates'): 4, ('fates', 'of'): 2, ('of', 'fates'): 2, ('fates', 'invisible'): 2, ('invisible', 'fates'): 2, ('fates', 'who'): 2, ('who', 'fates'): 2, ('fates', 'put'): 2, ('put', 'fates'): 2, ('fates', 'down'): 2, ('down', 'fates'): 2, ('fates', 'managers'): 2, ('managers', 'fates'): 2, ('fates', 'for'): 2, ('for', 'fates'): 2, ('fates', 'stage'): 2, ('stage', 'fates'): 2, ('fates', ','): 4, (',', 'fates'): 4, ('fates', 'me'): 2, ('me', 'fates'): 2, ('fates', 'constant'): 2, ('constant', 'fates'): 2, ('fates', 'police'): 2, ('police', 'fates'): 2, ('fates', 'ha

In [69]:
print(weighting_dic['10', 'or'])
print(weighting_dic['or', '10'])

0.053182958969449884
0.053182958969449884


In [70]:
test = random.choice(window_data)
print(test)
try:
    print(X_ik[(test[0], test[1])] == X_ik[(test[1], test[0])])
except:
    1

('sacred', 'any')
True


### Prepare train data

In [71]:
u_p = [] # center vec
v_p = [] # context vec
co_p = [] # log(x_ij)
weight_p = [] # f(x_ij)

for pair in window_data: 
    u_p.append(prepare_word(pair[0], word2index).view(1, -1))
    v_p.append(prepare_word(pair[1], word2index).view(1, -1))
    
    try:
        cooc = X_ik[pair]
    except:
        cooc = 1

    co_p.append(torch.log(Variable(FloatTensor([cooc]))).view(1, -1))
    weight_p.append(Variable(FloatTensor([weighting_dic[pair]])).view(1, -1))
                                  
train_data = list(zip(u_p, v_p, co_p, weight_p))
del u_p
del v_p
del co_p
del weight_p
print(train_data[0]) # tuple (center vec i, context vec j log(x_ij), weight f(w_ij))

(Variable containing:
 1059
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, Variable containing:
 240
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
, Variable containing:
1.00000e-02 *
  5.3183
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]
)


## Modeling 

<img src="../images/03.glove-objective.png">
<center>borrowed image from https://nlp.stanford.edu/pubs/glove.pdf</center>

In [0]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,projection_dim):
        super(GloVe,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, projection_dim) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
        initrange = (2.0 / (vocab_size + projection_dim))**0.5 # Xavier init
        self.embedding_v.weight.data.uniform_(-initrange, initrange) # init
        self.embedding_u.weight.data.uniform_(-initrange, initrange) # init
        self.v_bias.weight.data.uniform_(-initrange, initrange) # init
        self.u_bias.weight.data.uniform_(-initrange, initrange) # init
        
    def forward(self, center_words, target_words, coocs, weights):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        
        center_bias = self.v_bias(center_words).squeeze(1) 
        target_bias = self.u_bias(target_words).squeeze(1) 
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) 
        # Bx1xD * BxDx1 => Bx1x1 => Bx1
        
        loss = weights*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)
    
    def prediction(self, inputs):
        v_embeds = self.embedding_v(inputs) # B x 1 x D
        u_embeds = self.embedding_u(inputs) # B x 1 x D
                
        return v_embeds+u_embeds # final embed

## Train 

In [0]:
EMBEDDING_SIZE = 50
BATCH_SIZE = 256
EPOCH = 50

In [0]:
losses = []
model = GloVe(len(word2index), EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [76]:
for epoch in range(EPOCH):
    for i,batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets, coocs, weights = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        coocs = torch.cat(coocs)
        weights = torch.cat(weights)
        model.zero_grad()

        loss = model(inputs, targets, coocs, weights)
        
        loss.backward()
        optimizer.step()
    
        losses.append(loss.data.tolist()[0])
    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch, np.mean(losses)))
        losses = []

Epoch : 0, mean_loss : 217.72
Epoch : 10, mean_loss : 2.74
Epoch : 20, mean_loss : 0.56
Epoch : 30, mean_loss : 0.13
Epoch : 40, mean_loss : 0.05


## Test 

In [0]:
def word_similarity(target, vocab):
    if USE_CUDA:
        target_V = model.prediction(prepare_word(target, word2index))
    else:
        target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: 
            continue
        
        if USE_CUDA:
            vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        else:
            vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0] 
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [78]:
test = random.choice(list(vocab))
test

'unwieldy'

In [79]:
word_similarity(test, vocab)

[['joy', 0.9673048853874207],
 ['express', 0.9609348773956299],
 ['sailors', 0.7666824460029602],
 ['icelandic', 0.722845733165741],
 ['exercise', 0.720321536064148],
 ['virtue', 0.7200307846069336],
 ['gloom', 0.7103403210639954],
 ['breedeth', 0.6963592767715454],
 ['competent', 0.6893061399459839],
 ['judgmatically', 0.6890193819999695]]

## TODO

* Use <a href="https://docs.scipy.org/doc/scipy/reference/sparse.html">sparse-matrix</a> to build co-occurence matrix for memory efficiency

## Suggested Readings

* <a href="http://ruder.io/word-embeddings-2017/?utm_campaign=Revue%20newsletter&utm_medium=Newsletter&utm_source=The%20Wild%20Week%20in%20AI">Word embeddings in 2017: Trends and future directions</a>